In [71]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [12]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

/var/folders/4m/bmr6pj2s4k996yk6vfh6_1y40000gn/T/ipykernel_35992/1335968433.py:1: DtypeWarning: Columns (11,28,46,51,57,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("train.csv")
/var/folders/4m/bmr6pj2s4k996yk6vfh6_1y40000gn/T/ipykernel_35992/1335968433.py:2: DtypeWarning: Columns (45,56) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv("test.csv")


In [13]:
X = train.drop(columns=["target"])
y = train["target"]

In [14]:
def fix_commas(df):
    return df.apply(lambda col: col.astype(str).str.replace(",", ".", regex=False))

In [15]:
X = fix_commas(X)
test_fixed = fix_commas(test)

In [16]:
X = X.apply(pd.to_numeric, errors='coerce')
test_fixed = test_fixed.apply(pd.to_numeric, errors='coerce')

In [17]:
X = X.replace([np.inf, -np.inf, -1], np.nan)
test_fixed = test_fixed.replace([np.inf, -np.inf, -1], np.nan)

In [18]:
print(X.shape)
print(test_fixed.shape)

(90329, 65)
(10037, 65)


In [78]:
model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.12,
    depth=16,
    od_type="Iter",
    l2_leaf_reg=7,
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=2004,
    verbose=10
)

In [79]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=2004)

In [80]:
model.fit(X_train, y_train, eval_set=(X_val, y_val))

0:	test: 0.6460405	best: 0.6460405 (0)	total: 5.33s	remaining: 8m 47s
10:	test: 0.7010910	best: 0.7014308 (9)	total: 28.8s	remaining: 3m 52s
20:	test: 0.7128670	best: 0.7128670 (20)	total: 59.9s	remaining: 3m 45s
30:	test: 0.7145642	best: 0.7160456 (25)	total: 1m 28s	remaining: 3m 16s
40:	test: 0.7086670	best: 0.7160456 (25)	total: 2m 5s	remaining: 3m 1s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.716045635
bestIteration = 25

Shrink model to first 26 iterations.


In [81]:
pred_label = model.predict(test_fixed)

In [82]:
answers = pd.DataFrame({
    "customer": test["customer"],   # обязательно по порядку!
    "target": pred_label
})

In [83]:
answers

,customer,target
0,119156836,0
1,46948414,0
2,85702891,0
3,84487348,0
4,122306861,0
...,...,...
10032,83253522,0
10033,39917601,0
10034,23787469,0
10035,146734036,0


In [84]:
answers.to_csv("answers.csv", index=False)